##### Abstraction to make task easier and intuitive

## 1. Before abstraction

In [1]:
import tensorflow as tf

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    h = tf.nn.relu(conv2d(input, W) + b)
    hp = max_pool_2x2(h)
    return hp

x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

h1 = conv_layer(x_image, shape=[5, 5, 1, 32])
h2 = conv_layer(h1, shape=[5, 5, 32, 64])
h3 = conv_layer(h2, shape=[5, 5, 64, 32])

##### Error prone

## 2. With abstraction library 

#### e.g. ( tf.contrib.learn, TFLearn, contrib.slim, Keras)

'contrib.learn', 'contrib.slim' and ' contrib.keras. It is not needed to install and It means 'contributed' to tf

### 2.1 contrib.learn

Using like, DNNClassifier(), learn, fit, evaluate and predict are available.

In [10]:
from tensorflow.contrib import learn

import sys
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#### Data load

In [11]:
DATA_DIR = 'C:/tmp/data' if not 'win32' in sys.platform else "C:\\tmp\\data"
data = input_data.read_data_sets(DATA_DIR, one_hot=False)
x_data, y_data = data.train.images, data.train.labels.astype(np.int32)
x_test, y_test = data.test.images, data.test.labels.astype(np.int32)

Extracting C:\tmp\data\train-images-idx3-ubyte.gz
Extracting C:\tmp\data\train-labels-idx1-ubyte.gz
Extracting C:\tmp\data\t10k-images-idx3-ubyte.gz
Extracting C:\tmp\data\t10k-labels-idx1-ubyte.gz


#### Model

In [12]:
EPOCH_NUM = 2000
MINIBATCH_SIZE = 128

feature_columns = learn.infer_real_valued_columns_from_input(x_data)

dnn = learn.DNNClassifier(feature_columns=feature_columns,
                          hidden_units=[200],
                          n_classes=10,
                          optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.2))

dnn.fit(x=x_data, y=y_data, steps=EPOCH_NUM, batch_size=MINIBATCH_SIZE)

test_acc = dnn.evaluate(x=x_test, y=y_test, steps=1)["accuracy"]
print('test accuracy: {}'.format(test_acc))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DF20CBDB38>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\R\\Temp\\tmpwpr8mvvh'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\R\Temp

### ** Tip. Useful function, FeatureColumn

layers.sparse_column_with_keys()

layers.sparse_column_with_hash_bucket()

layers.crossed_column()

layers.bucketized_column()